# Tugas

### **Smartphone (PyDroid3)**
| Komponen | Spesifikasi | Pengaruh |
|-----------|-------------|-----------|
| **Perangkat** | Redmi Note 10S | — |
| **Chipset** | MediaTek G95 | Menentukan kecepatan komputasi numerik |
| **CPU** | Octa-core Max 2.05GHz | Menghitung jarak antar vektor (Euclidean distance) |
| **RAM** | 8 GB  | Membatasi ukuran dataset yang bisa dimuat |
| **Penyimpanan** | 128 GB | Memengaruhi kecepatan load dataset |
| **Sistem Operasi** | Android 13 (MIUI 14) dengan PyDroid3 | Interpreter Python ARM64 |
| **Arsitektur** | ARMv8-A (64-bit) | Berpengaruh pada vectorization NumPy |

<br>

## Hasil Eksperimen

| Algoritma     | HP (detik) | Colab (detik) | Rasio HP ÷ Colab          | Lebih Cepat di                     |
| ------------- | ---------- | ------------- | ------------------------- | ---------------------------------- |
| **Exact NN**  | 1.172      | 2.294         | 1.172 ÷ 2.294 ≈ **0.51×** | **Colab** (≈2× lebih lambat di HP) |
| **Annoy**     | 1.588      | 1.442         | 1.588 ÷ 1.442 ≈ **1.10×** | **HP** (≈1.1× lebih cepat)         |
| **HNSW**      | 8.928      | 7.078         | 8.928 ÷ 7.078 ≈ **1.26×** | **HP** (≈1.3× lebih cepat)         |
| **FAISS IVF** | —          | 0.399         | —                         | **Colab** (tidak diuji di HP)      |

<br>

In [ ]:
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors
from annoy import AnnoyIndex
import hnswlib
import faiss
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Contoh dataset kecil untuk testing
# -------------------------------
np.random.seed(42)
n_samples = 10000   # jumlah database vector
d = 128             # dimensi
X = np.random.random((n_samples, d)).astype('float32')

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

# -------------------------------
# Exact NN (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=d)
p_hnsw.init_index(max_elements=n_samples, ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, _ = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()

d = X_scaled.shape[1]
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index_faiss = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 10
# Fix: Search using the same data that was added to the index (X_scaled)
_, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")

# -------------------------------
# Tampilkan ringkasan waktu
# -------------------------------
print("\n=== Ringkasan Waktu (detik) ===")
print(f"Exact NN : {time_exact:.3f}")
print(f"Annoy    : {time_annoy:.3f}")
print(f"HNSW     : {time_hnsw:.3f}")
print(f"FAISS    : {time_faiss:.3f}")

Exact NN done in 2.294 s
Annoy done in 1.442 s
HNSW done in 7.078 s
FAISS IVF done in 0.399 s

=== Ringkasan Waktu (detik) ===
Exact NN : 2.294
Annoy    : 1.442
HNSW     : 7.078
FAISS    : 0.399
